In [2]:
import numpy as np
from os import listdir
import os
from os.path import isfile, join
from pathlib import Path


folder_path = "dataset/topo/embeddings/dnabert"


# list of files in the folder without the extension
IDs = [Path(f).stem for f in listdir(folder_path) if isfile(join(folder_path, f))]


embeddings_matrix = []


for id in IDs:

    # load the file as a numpy array
    raw_embedding = None
    try:
        raw_embedding = np.load(folder_path + "/" + id + ".npy")
        print(raw_embedding.shape)
    except:
        raw_embedding = np.load(folder_path + "/" + id + ".npy", allow_pickle=True)
        print(f"Error while loading the embedding of sequence {id}")
        # os.remove(folder_path + "/" + id + ".npy")
        print(raw_embedding.shape)
        

(3, 445, 768)
(1, 416, 768)
(2, 264, 768)
(2, 254, 768)
(1, 362, 768)
(3, 378, 768)
(1, 400, 768)
(1, 258, 768)
(1, 221, 768)
(1, 241, 768)
(1, 462, 768)
(1, 162, 768)
(1, 340, 768)
(1, 223, 768)
(1, 328, 768)
(1, 328, 768)
(1, 143, 768)
(1, 488, 768)
(1, 223, 768)
(1, 80, 768)
(1, 462, 768)
(1, 343, 768)
(1, 277, 768)
(2, 297, 768)
(1, 273, 768)
(1, 364, 768)
(2, 460, 768)
(1, 367, 768)
(2, 382, 768)
(1, 442, 768)
(1, 272, 768)
(2, 410, 768)
(2, 458, 768)
(2, 439, 768)
(1, 353, 768)
(2, 451, 768)
(2, 303, 768)
(1, 264, 768)
(2, 341, 768)
(1, 195, 768)
(2, 366, 768)
(2, 311, 768)
(2, 271, 768)
(3, 374, 768)
(1, 266, 768)
(1, 89, 768)
(2, 386, 768)
(1, 141, 768)
(1, 181, 768)
(1, 348, 768)
(2, 263, 768)
(1, 139, 768)
(1, 243, 768)
(1, 340, 768)
(1, 351, 768)
(1, 428, 768)
(1, 182, 768)
(1, 453, 768)
(2, 427, 768)
(1, 411, 768)
(1, 251, 768)
(2, 418, 768)
(1, 344, 768)
(1, 319, 768)
(2, 346, 768)
(1, 364, 768)
(2, 257, 768)
(2, 411, 768)
(1, 395, 768)
(1, 350, 768)
(3, 392, 768)
(3, 504,

In [3]:
import numpy as np
from os import listdir
from os.path import isfile, join
from pathlib import Path
from Bio import SeqIO


FASTA_FILE_PATH = "dataset/covid19/covid19_proteins.fasta"
lens = []

for count, seqrecord in enumerate(SeqIO.parse(FASTA_FILE_PATH, "fasta")):

    lens.append(len(str(seqrecord.seq)))

print(np.mean(lens))
print(np.max(lens))



885.0909090909091
7096


In [31]:
# export the geneIDs given a xml uniprot file
from Bio import SeqIO

records = list(SeqIO.parse("dataset/emoglobina/emoglobina.xml", "uniprot-xml"))
geneIDs = []

xml_names = []
for record in records:
    for i in record.dbxrefs:
        if i.startswith("GeneID:"):
            geneID = i.split(":")[1]
            geneIDs.append(geneID)
            break

with open("emoglobina_ids.txt", 'w') as f:
    for i in geneIDs:
        f.write(i + "\n")



In [5]:
# given a genbank(full) file, for each proteome select the superset proteins (CDS) and add them to a fasta file

from Bio import SeqIO
import Bio

no_ids = 0

def to_simple_location(f_list):
    for feature in f_list: 
        if type(feature.location) == Bio.SeqFeature.CompoundLocation:
            feature.location = (int(feature.location.parts[0].start) , int(feature.location.parts[1].end))
        else:
            feature.location = (int(feature.location.start), int(feature.location.end))

def remove_subsets(f_list):
    for i in range(len(f_list)):
        for j in range(i+1, len(f_list)):
            if f_list[i].location[0] <= f_list[j].location[0] and f_list[i].location[1] >= f_list[j].location[1]:
                # print(f'removed protein {f_list[j].qualifiers["protein_id"]}')
                f_list.pop(j)
                break
            elif f_list[j].location[0] <= f_list[i].location[0] and f_list[j].location[1] >= f_list[i].location[1]:
                # print(f'removed protein {f_list[i].qualifiers["protein_id"]}')
                f_list.pop(i)
                break
            else:
                continue
        break
    return f_list


gb_records = list(SeqIO.parse('dataset/covid19/covid19.gb','genbank'))
final_proteins = []

for gb_record in gb_records:
    CDS_features = []

    for feature in gb_record.features:
        if feature.type == 'CDS':
            CDS_features.append(feature) 

    to_simple_location(CDS_features)

    # print(len(CDS_features))
    remove_subsets(CDS_features)
    # print(len(CDS_features))

    for feature in CDS_features:
        # print(feature.location, feature.qualifiers['protein_id'], feature.qualifiers['translation'])

        try:
            final_proteins.append((feature.qualifiers['protein_id'], feature.qualifiers['translation']))
        except:
            no_ids +=1


print(len(final_proteins))
print(no_ids)  
 

# output_file = f"dataset/covid19/covid19_proteins.fasta"

# with open(output_file, 'w') as fasta_file:
#     for entry in final_proteins:
#         description, sequence = entry
#         fasta_file.write(f'>{description[0]}\n{sequence[0]}\n')




847
0


In [1]:
from Bio import SeqIO

gb_records = list(SeqIO.parse('dataset/covid19/covid19.gb','genbank'))


for gb_record in gb_records:
    print(gb_record.id)

MW969758.1
MW969757.1
MW969756.1
MW969755.1
MW969754.1
MW969753.1
MW969752.1
MW646603.1
MW646602.1
MW646601.1
MW646600.1
MW646599.1
MW646598.1
MW646597.1
MW646431.1
MW646430.1
MW646429.1
MW646428.1
MW646427.1
MW646426.1
MW646425.1
MW646424.1
MW646423.1
MW642258.1
MW642257.1
MW642254.1
MW642253.1
MZ242711.1
MZ242695.1
MT994881.1
MT994849.1
MT994632.1
MT994286.1
MT889692.1
MT675517.1
MT675289.1
MT506707.1
OK081890.1
OK081889.1
OK081888.1
OK081887.1
OK081886.1
MW854297.1
MW852494.1
MW185827.1
MW185826.1
MW185825.1
MW185824.1
MW185823.1
MW004168.1
MT703677.1
MT679208.1
MT679206.1
MT679205.1
MT679202.1
MT679201.1
MT679198.1
MT682732.1
MT447176.1
MT447175.1
MT447174.1
MT447173.1
MT447172.1
MT447171.1
MT447169.1
MT447166.1
MT447165.1
MT447163.1
MT447162.1
MT447161.1
MT447159.1
MT447158.1
MT447157.1
MT447156.1
MT447155.1
MT447154.1
MT300186.2


In [5]:
from Bio import SeqIO
import os
import numpy as np 

path = '/storagenfs/m.tolloso/BioEmbedding/dataset/covid19/embeddings/alphafold'

# for each file in the folder 'path'

ids = os.listdir(path)


tensor1 = np.load(path + '/' + ids[0])
tensor2 = np.load(path + '/' + ids[3])

print(tensor1.shape)
print(tensor2.shape)

# concat the tensors over the second axis

concat = np.concatenate((tensor1, tensor2), axis=1)

print(concat.shape)

(1, 61, 384)
(1, 222, 384)
(1, 283, 384)
